# 1. investigate the counts per research grade and location

In [ ]:
from pyinaturalist import get_observations
import pandas as pd
import time

def get_counts(classes, place_id):

    print(f"{'ID':<4} {'Class Name':<20} {'Total Imgs (NC)'}")
    print("-" * 40)

    results = []

    for class_id, info in classes.items():
        total_count = 0
        
        # Some classes have multiple families (e.g., Spiders), so we loop through IDs
        for taxon_id in info['taxon_ids']:
            
            # KEY QUERY: Fetch counts for Research Grade obs in NC
            response = get_observations(
                taxon_id=taxon_id,
                place_id=place_id,
                quality_grade="research", # Only high quality data
                
                # SPECIAL FILTER FOR CATERPILLARS (Class 7)
                # term_id=1 (Life Stage), term_value_id=6 (Larva)
                term_id=1 if class_id == 7 else None,
                term_value_id=6 if class_id == 7 else None,

                count_only=True
            )
            
            # The API returns specific species counts; we sum them up for the whole family
            total_count += response['total_results']
            
            # Be nice to the API
            time.sleep(0.5)

        print(f"{class_id:<4} {info['name']:<20} {total_count}")
        results.append({'Class': info['name'], 'Count': total_count})

In [32]:
# IDs found via iNaturalist taxonomy search
classes = {
    0:  {'name': 'Venomous Spiders',   'taxon_ids': [47370, 48140]}, # Latrodectus (not Theridiidae as this includes non harmful), Sicariidae
    1:  {'name': 'Ticks',              'taxon_ids': [51673]},        # Ixodidae 
    2:  {'name': 'Mosquitoes',         'taxon_ids': [52134]},        # Culicidae
    3:  {'name': 'Stinging Wasps',     'taxon_ids': [52747]},        # Vespidae
    4:  {'name': 'Bees',               'taxon_ids': [47221]},        # Apidae
    5:  {'name': 'Fire Ants',          'taxon_ids': [67597]},        # Genus Solenopsis
    6:  {'name': 'Assassin Bugs',      'taxon_ids': [48959]},        # Reduviidae
    7:  {'name': 'Venomous Cats.',     'taxon_ids': [84186, 84165]}, # Megalopygidae, Limacodidae 
    8:  {'name': 'Blister Beetles',    'taxon_ids': [59510]},        # Meloidae 
    9:  {'name': 'Scorpions',          'taxon_ids': [52572]},        # Vaejovidae 
    10: {'name': 'Horse/Deer Flies',   'taxon_ids': [47821]},        # Tabanidae 
    11: {'name': 'Centipedes',         'taxon_ids': [49556]}         # Order Chilopoda
}

In [33]:
get_counts(classes, 30) # NC

ID   Class Name           Total Imgs (NC)
----------------------------------------
0    Venomous Spiders     1187
1    Ticks                1291
2    Mosquitoes           1483
3    Stinging Wasps       16266
4    Bees                 38242
5    Fire Ants            1535
6    Assassin Bugs        8209
7    Venomous Cats.       1977
8    Blister Beetles      755
9    Scorpions            26
10   Horse/Deer Flies     652
11   Centipedes           2135


In [34]:
get_counts(classes, 1) # USA

ID   Class Name           Total Imgs (NC)
----------------------------------------
0    Venomous Spiders     48043
1    Ticks                47756
2    Mosquitoes           30420
3    Stinging Wasps       414925
4    Bees                 1171609
5    Fire Ants            23933
6    Assassin Bugs        150786
7    Venomous Cats.       23665
8    Blister Beetles      36069
9    Scorpions            26067
10   Horse/Deer Flies     15492
11   Centipedes           42634


In [35]:
get_counts(classes, None) # world

ID   Class Name           Total Imgs (NC)
----------------------------------------
0    Venomous Spiders     77358
1    Ticks                65751
2    Mosquitoes           55342
3    Stinging Wasps       773799
4    Bees                 2036818
5    Fire Ants            29182
6    Assassin Bugs        232046
7    Venomous Cats.       36864
8    Blister Beetles      89148
9    Scorpions            28889
10   Horse/Deer Flies     35505
11   Centipedes           92593
